In [47]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from matplotlib import pyplot as plt
import os
import shutil
import argparse

In [35]:
#Dataset_dir = '\\Users\\Sadie Nathan\\Documents\\GitHub\\Term 2\\Mini Project\\compounded_data' #"C:\Users\Sadie Nathan\Documents\GitHub\Term 2\Mini Project\compounded_data"
#Output_dir = 'output'
#try:
#        os.makedir(Dataset_dir)
#        os.makedir(Output_dir)
#except:
#    pass

In [48]:
directory = "compounded_data"
 
# Parent Directory path
parent_dir = "\\Users\\Sadie Nathan\\Documents\\GitHub\\Term 2\\Mini Project\\"
 
# Path
path = os.path.join(parent_dir, directory)
 

try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" %directory)
except OSError as error:
    print("Directory '%s' can not be created")

Directory 'compounded_data' created successfully


In [49]:
dir_list = os.listdir(path) 
dir_list[:5] #for some reason it isn't able to show the full number of images

['data (1).jpg',
 'data (10).jpg',
 'data (100).jpg',
 'data (1000).jpg',
 'data (10000).jpg']

In [50]:
pip install split-folders

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [53]:
import split_folders # or import splitfolders
input_folder = path
output = "\\Users\\Sadie Nathan\\Documents\\GitHub\\Term 2\\Mini Project\\Output" #where you want the split datasets saved. one will be created if it does not exist or none is set

split_folders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

ModuleNotFoundError: No module named 'split_folders'

In [43]:
def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def main(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))

    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)


def parse_args():
  parser = argparse.ArgumentParser(description="Dataset divider")
  parser.add_argument("--data_path", required=True,
    help="Path to data")
  parser.add_argument("--test_data_path_to_save", required=True,
    help="Path to test data where to save")
  parser.add_argument("--train_ratio", required=True,
    help="Train ratio - 0.7 means splitting data in 70 % train and 30 % test")
  return parser.parse_args()

if __name__ == "__main__":
  args = parse_args()
  main(args.data_path, args.test_data_path_to_save, float(args.train_ratio))

usage: ipykernel_launcher.py [-h] --data_path DATA_PATH --test_data_path_to_save TEST_DATA_PATH_TO_SAVE --train_ratio
                             TRAIN_RATIO
ipykernel_launcher.py: error: the following arguments are required: --data_path, --test_data_path_to_save, --train_ratio


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
image_dims = (28, 28, 1)

In [25]:
latent_dim = 2
encoder = Sequential([
    layers.Input(shape = image_dims),
    layers.Conv2D(32, 3, activation="relu", strides=2, padding="same"),
    layers.Conv2D(64, 3, activation="relu", strides=2, padding="same"),
    layers.Flatten(),
    layers.Dense(16, activation="relu"),
    ##Output is two numbers
    layers.Dense(latent_dim)
])

In [26]:
encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 16)                50192     
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 69,042
Trainable params: 69,042
Non-trainable params: 0
_________________________________________________________________


In [27]:
image_dims = (28, 28, 3)

In [28]:
quarter = int(image_dims[0]/4)

In [29]:
decoder = Sequential([
    keras.Input(shape=(latent_dim,)),
    layers.Dense(quarter * quarter * 64, activation="relu"),
    layers.Reshape((quarter, quarter, 64)),
    layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same"),
    layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same"),
    layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")
])

In [30]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        289       
 ranspose)                                                       
                                                      

In [31]:
auto_encoder = Sequential([encoder, decoder])

In [39]:
X = path.iloc[:, :-1]
y = path.iloc[:, -1]

AttributeError: 'str' object has no attribute 'iloc'

In [38]:
(x_train, _), (x_test, _) = path
##Join test and train sets together
mnist_digits = np.concatenate([x_train, x_test], axis=0)
##Normalise down to 0-1
mnist_digits = np.expand_dims(path, -1).astype("float32") / 255

ValueError: too many values to unpack (expected 2)